# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




In [1]:
# Make sure you are using Python 3.8-3.11 and have pip installed
# spin up a venv
# python -m venv ./env
# source ./env/bin/activate

SyntaxError: invalid syntax (592493819.py, line 3)

In [1]:
# first installed the dependencies:

In [2]:
%%capture
!pip install dlt[duckdb]

In [50]:
import duckdb
import dlt

# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [4]:
# QUESTION 1

limit = 5
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum += sqrt_value
print("sum of the outputs of the generator for limit=5 :",  sum)

sum of the outputs of the generator for limit=5 : 8.382332347441762


In [5]:
# QUESTION 2

limit = 13
generator = square_root_generator(limit)

i=1
for sqrt_value in generator:
    print(f"[{i}]:", sqrt_value)
    i += 1

[1]: 1.0
[2]: 1.4142135623730951
[3]: 1.7320508075688772
[4]: 2.0
[5]: 2.23606797749979
[6]: 2.449489742783178
[7]: 2.6457513110645907
[8]: 2.8284271247461903
[9]: 3.0
[10]: 3.1622776601683795
[11]: 3.3166247903554
[12]: 3.4641016151377544
[13]: 3.605551275463989


In [6]:
# QUESTION 2 ALTERNATIVE i

limit = 13
generator = square_root_generator(limit)

i=0
for sqrt_value in generator:
    i += 1

print("Thirteenth square root value:", sqrt_value)

Thirteenth square root value: 3.605551275463989


In [7]:
# QUESTION 2 ALTERNATIVE ii

limit = 13
generator = square_root_generator(limit)

for _ in range(12):
    next(generator)

thirteenth_value = next(generator)
print(thirteenth_value)

3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [68]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [114]:
# QUESTION 3

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

# we can load any generator to a table at the pipeline destnation as follows:
info = pipeline.run(people_1(),
                						table_name="people_table_append",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

# set up a connection
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# sum of ages from people_1() generator
sum_of_ages = conn.execute("SELECT SUM(Age) FROM people.people_table_append").df()
display(sum_of_ages)

# we can load the next generator to the same or to a different table.
info = pipeline.run(people_2(),
										table_name="people_table_append",
										write_disposition="append")
print(info)

# sum of ages from people_1() + appended people_2() generator
sum_of_ages_all_append = conn.execute("SELECT SUM(Age) FROM people.people_table_append").df()
display(sum_of_ages_all_append)

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Pipeline dlt_ipykernel_launcher load step completed in 0.27 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////workspaces/de-zoom-by-dimi/data-engineering-zoomcamp-dimi/workshop-1-data-ingestion/my-workshop-1-hw-answers/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708292795.2842348 is LOADED and contains no failed jobs


,sum(Age)
0,140.0


Pipeline dlt_ipykernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////workspaces/de-zoom-by-dimi/data-engineering-zoomcamp-dimi/workshop-1-data-ingestion/my-workshop-1-hw-answers/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708292795.9973338 is LOADED and contains no failed jobs


,sum(Age)
0,353.0


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_table_append │
└─────────────────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [116]:
# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

# we can load any generator to a table at the pipeline destnation as follows:
info = pipeline.run(people_1(),
                						table_name="people_table_merge",
										write_disposition="replace",
                                        primary_key="ID")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

# we can load the next generator to the same or to a different table.
info = pipeline.run(people_2(),
										table_name="people_table_merge",
										write_disposition="merge",
                                        primary_key="ID")

print(info)

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

sum_of_ages_all_merged = conn.execute("SELECT SUM(Age) FROM people_table_merge").df()
display(sum_of_ages_all_merged)

Pipeline dlt_ipykernel_launcher load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////workspaces/de-zoom-by-dimi/data-engineering-zoomcamp-dimi/workshop-1-data-ingestion/my-workshop-1-hw-answers/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708292830.7959056 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.41 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////workspaces/de-zoom-by-dimi/data-engineering-zoomcamp-dimi/workshop-1-data-ingestion/my-workshop-1-hw-answers/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708292831.4635358 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_table_append │
│ people_table_merge  │
└─────────────────────┘

,sum(Age)
0,266.0


In [ ]:
# 2 SEE THE DATASET & TABLES

# # set up a connection
# conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# # Set the dataset name
# dataset_name = "people"  # Change this to your dataset name

# # Get the list of tables in the dataset
# tables = conn.execute(f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{dataset_name}'").fetchall()

# # Print the content of each table
# for table in tables:
#     table_name = table[0]
#     # Select all data from the table and fetch it
#     result = conn.execute(f"SELECT * FROM {dataset_name}.{table_name}").fetch_df()
#     # Print table name
#     print(f"Table: {table_name}")
#     # Print table data
#     print(result)

# # Close the connection
# conn.close()

# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX